# Seattle Open Data Portal for Data Analytics on Seattle

## Queries different Seattle data repositories to get the latest news on health

In [113]:
# import libraries
import requests
import urllib
import webbrowser
import json
import sys
import numpy
import matplotlib.pyplot as plt
import hvplot.pandas 
import sklearn
from dotenv import load_dotenv
import pandas as pd
import time
import datetime
import geopandas as gpd

from socrata.authorization import Authorization
from socrata import Socrata
import os
import argparse

In [114]:
# Load environment variables
load_dotenv()

True

In [115]:
# retrieve environment email and password
socrata_email = os.getenv("SOCRATA_EMAIL")
socrata_password = os.getenv("SOCRATA_PASSWORD")

# Check is loaded into variables
display(type(socrata_email))
display(type(socrata_password))

str

str

In [116]:
auth = Authorization(
    "data.seattle.gov",
    socrata_email,
    socrata_password
)

In [117]:
# create an HTTP Basic request with endpoint
# Data for crime in seattle based on location
data_url = 'https://data.seattle.gov/resource/kzjm-xkqj.json'
response = requests.get(data_url, auth=(socrata_email, socrata_password))

In [118]:
# turn response to json object
response.json()[0]

{'address': '2132 N Northgate Way',
 'type': 'Auto Fire Alarm',
 'datetime': '2023-01-30T16:29:00.000',
 'latitude': '47.70866',
 'longitude': '-122.333564',
 'report_location': {'type': 'Point', 'coordinates': [-122.333564, 47.70866]},
 'incident_number': 'F230012870',
 ':@computed_region_ru88_fbhk': '46',
 ':@computed_region_2day_rhn5': '134',
 ':@computed_region_cyqu_gs94': '1',
 ':@computed_region_kuhn_3gp2': '34',
 ':@computed_region_q256_3sug': '19582'}

In [119]:
# design a dataframe with just the datetime, type, latitude and longitude
test_list = [
    response.json()[0]['datetime'],
    response.json()[0]['type'],
    response.json()[0]['latitude'],
    response.json()[0]['longitude']
]

test_list

['2023-01-30T16:29:00.000', 'Auto Fire Alarm', '47.70866', '-122.333564']

In [120]:
# put data into dataframe
crime_df = pd.DataFrame.from_records(response.json())

# set datetime as index
crime_df = crime_df.set_index('datetime')
display(crime_df.head(10))
display(crime_df.tail(10))

,address,type,latitude,longitude,report_location,incident_number,:@computed_region_ru88_fbhk,:@computed_region_2day_rhn5,:@computed_region_cyqu_gs94,:@computed_region_kuhn_3gp2,:@computed_region_q256_3sug
datetime,,,,,,,,,,,
2023-01-30T16:29:00.000,2132 N Northgate Way,Auto Fire Alarm,47.70866,-122.333564,"{'type': 'Point', 'coordinates': [-122.333564,...",F230012870,46,134,1,34,19582
2023-01-30T16:29:00.000,501 Rainier Ave S,Aid Response,47.598322,-122.3134,"{'type': 'Point', 'coordinates': [-122.3134, 4...",F230012869,28,NaN,NaN,18,17919
2023-01-30T16:28:00.000,1930 Post Al,Aid Response,47.610181,-122.342117,"{'type': 'Point', 'coordinates': [-122.342117,...",F230012868,5,NaN,NaN,30,18081
2023-01-30T16:21:00.000,3051 Harbor Ave Sw,1RED 1 Unit,47.576495,-122.371373,"{'type': 'Point', 'coordinates': [-122.371373,...",F230012867,12,NaN,NaN,49,18385
2023-01-30T16:18:00.000,4915 25th Ave Ne,4RED - 2 + 1 + 1,47.664451,-122.300909,"{'type': 'Point', 'coordinates': [-122.300909,...",F230012864,53,NaN,NaN,48,18383
2023-01-30T16:18:00.000,843 Hiawatha Pl S,Aid Response,47.595241,-122.309737,"{'type': 'Point', 'coordinates': [-122.309737,...",F230012863,43,NaN,NaN,18,17919
2023-01-30T16:16:00.000,32nd Ave Sw / Sw Trenton St,MVI - Motor Vehicle Incident,47.524659,-122.373948,"{'type': 'Point', 'coordinates': [-122.373948,...",F230012862,54,NaN,NaN,14,19581
2023-01-30T16:09:00.000,36th Ave S / S Adams St,Illegal Burn,47.565398,-122.287386,"{'type': 'Point', 'coordinates': [-122.287386,...",F230012861,21,NaN,NaN,43,18800
2023-01-30T16:07:00.000,4540 8th Ave Ne,Aid Response,47.66167,-122.319564,"{'type': 'Point', 'coordinates': [-122.319564,...",F230012860,60,NaN,NaN,38,18383


,address,type,latitude,longitude,report_location,incident_number,:@computed_region_ru88_fbhk,:@computed_region_2day_rhn5,:@computed_region_cyqu_gs94,:@computed_region_kuhn_3gp2,:@computed_region_q256_3sug
datetime,,,,,,,,,,,
2023-01-27T13:26:00.000,3814 S Cloverdale St,Low Acuity Response,47.526112,-122.284612,"{'type': 'Point', 'coordinates': [-122.284612,...",F230011590,58,NaN,NaN,44,18800
2023-01-27T13:23:00.000,3015 Nw 54th St,Aid Response,47.667821,-122.396176,"{'type': 'Point', 'coordinates': [-122.396176,...",F230011588,4,NaN,NaN,1,18386
2023-01-27T13:20:00.000,330 19th Ave E,Aid Response,47.620954,-122.307342,"{'type': 'Point', 'coordinates': [-122.307342,...",F230011587,35,NaN,NaN,6,18789
2023-01-27T13:06:00.000,6600 Roosevelt Way Ne,Aid Response,47.676554,-122.317373,"{'type': 'Point', 'coordinates': [-122.317373,...",F230011586,53,NaN,NaN,47,18792
2023-01-27T13:01:00.000,320 Nw 75th St,Automatic Fire Alarm Resd,47.683276,-122.360825,"{'type': 'Point', 'coordinates': [-122.360825,...",F230011585,23,NaN,NaN,23,18798
2023-01-27T12:55:00.000,2914 S Mcclellan St,Medic Response,47.578347,-122.295037,"{'type': 'Point', 'coordinates': [-122.295037,...",F230011583,38,NaN,NaN,42,17919
2023-01-27T12:53:00.000,4744 41st Ave Sw,Aid Response,47.560296,-122.38416,"{'type': 'Point', 'coordinates': [-122.38416, ...",F230011581,1,NaN,NaN,50,18795
2023-01-27T12:53:00.000,325 Westlake Ave N,Rubbish Fire,47.62116,-122.338463,"{'type': 'Point', 'coordinates': [-122.338463,...",F230011582,56,NaN,NaN,10,18390
2023-01-27T12:51:00.000,2870 S Hanford St,Low Acuity Response,47.574853,-122.295888,"{'type': 'Point', 'coordinates': [-122.295888,...",F230011580,10,NaN,NaN,42,17919


In [121]:
# trim dataframe to just type, latitude and longitude
trimmed_crime_df = crime_df.loc[:, ['type', 'latitude', 'longitude']]
trimmed_crime_df


,type,latitude,longitude
datetime,,,
2023-01-30T16:29:00.000,Auto Fire Alarm,47.70866,-122.333564
2023-01-30T16:29:00.000,Aid Response,47.598322,-122.3134
2023-01-30T16:28:00.000,Aid Response,47.610181,-122.342117
2023-01-30T16:21:00.000,1RED 1 Unit,47.576495,-122.371373
2023-01-30T16:18:00.000,4RED - 2 + 1 + 1,47.664451,-122.300909
...,...,...,...
2023-01-27T12:55:00.000,Medic Response,47.578347,-122.295037
2023-01-27T12:53:00.000,Aid Response,47.560296,-122.38416
2023-01-27T12:53:00.000,Rubbish Fire,47.62116,-122.338463


In [122]:
# investigate dataframe
trimmed_crime_df.dtypes

type         object
latitude     object
longitude    object
dtype: object

In [123]:
# check unique values
print(trimmed_crime_df['type'].unique())

['Auto Fire Alarm' 'Aid Response' '1RED 1 Unit' '4RED - 2 + 1 + 1'
 'MVI - Motor Vehicle Incident' 'Illegal Burn' 'Automatic Fire Alarm Resd'
 'Medic Response' 'Scenes Of Violence 7' 'Nurseline/AMR'
 'Triaged Incident' 'Low Acuity Response' 'Rescue Elevator'
 'Medic Response- Overdose' 'Aid Response Freeway' 'Bark Fire'
 'Aid Response Yellow' 'AFA4 - Auto Alarm 2 + 1 + 1' 'Car Fire W/Exp.'
 'Automatic Fire Alarm False' 'Encampment Fire'
 'Investigate Out Of Service' 'Unk Odor' 'Alarm Bell' 'Car Fire'
 'Rubbish Fire' 'Rescue Lock In/Out' 'Fire in Building' 'Natural Gas Odor'
 'Single Medic Unit' 'Hang-Up- Aid' 'Rescue Extrication'
 'Activated CO Detector' 'Low Acuity Referral'
 'Medic Response- 6 per Rule' 'Trans to AMR' 'EVENT - Special Event'
 'Medic Response- 7 per Rule' 'Wires Down' 'Dumpster Fire'
 'Mutual Aid- Engine' 'Electrical Problem' 'Water Rescue Response'
 'Natural Gas Leak' 'MVI Freeway' 'Poison Control' 'Dumpster Fire W/Exp.'
 'Car Fire Freeway' 'Brush Fire' 'MVI Medic' '

In [124]:
no_duplicate_df = trimmed_crime_df[~trimmed_crime_df.index.duplicated()]
no_duplicate_df = no_duplicate_df.reset_index()
no_duplicate_df


,datetime,type,latitude,longitude
0,2023-01-30T16:29:00.000,Auto Fire Alarm,47.70866,-122.333564
1,2023-01-30T16:28:00.000,Aid Response,47.610181,-122.342117
2,2023-01-30T16:21:00.000,1RED 1 Unit,47.576495,-122.371373
3,2023-01-30T16:18:00.000,4RED - 2 + 1 + 1,47.664451,-122.300909
4,2023-01-30T16:16:00.000,MVI - Motor Vehicle Incident,47.524659,-122.373948
...,...,...,...,...
887,2023-01-27T13:01:00.000,Automatic Fire Alarm Resd,47.683276,-122.360825
888,2023-01-27T12:55:00.000,Medic Response,47.578347,-122.295037
889,2023-01-27T12:53:00.000,Aid Response,47.560296,-122.38416
890,2023-01-27T12:51:00.000,Low Acuity Response,47.574853,-122.295888


In [125]:
# get value counts of types
value_counts = no_duplicate_df['type'].value_counts()
value_counts

Aid Response                    433
Medic Response                   77
Auto Fire Alarm                  57
Low Acuity Response              49
Triaged Incident                 33
Illegal Burn                     30
Nurseline/AMR                    29
Investigate Out Of Service       19
MVI - Motor Vehicle Incident     19
Medic Response- Overdose         17
Automatic Fire Alarm Resd        12
Rescue Elevator                  12
Rubbish Fire                      9
Aid Response Yellow               8
Medic Response- 7 per Rule        7
Activated CO Detector             7
Alarm Bell                        7
1RED 1 Unit                       6
Medic Response- 6 per Rule        4
Unk Odor                          4
AFA4 - Auto Alarm 2 + 1 + 1       4
MVI Freeway                       3
Aid Response Freeway              3
EVENT - Special Event             3
Fire in Building                  3
Scenes Of Violence 7              3
Encampment Fire                   3
Natural Gas Odor            

In [126]:
# plot geospatially
geo_plot = no_duplicate_df.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True, 
    color='type',
    size='',
    alpha=0.8,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    )

geo_plot


c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\silva\anaconda3\envs\dev\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (type)